# Barrier Option

## Down and Out Option 

In [128]:
from numpy import sqrt, exp, cumsum, sum, maximum, mean
from numpy.random import standard_normal
import numpy as np
from scipy.stats import norm, qmc

# Parameters
S0 = 100; T = 0.5; K = 110; r = 0.05; sigma = 0.5
M = 1000; dt = T / M; I = 10000 ; B=75
payoff=np.zeros(I)

In [129]:
# Simulate I paths with M time steps
S = S0 * exp(cumsum((r - 0.5 * sigma ** 2) * dt + sigma * sqrt(dt) * 
                          standard_normal((M, I)), axis=0 ))
for i in range(I):
    if min(S[:,i])<=B :
        payoff[i]=0 
    else :
        payoff[i]=max(S[-1:,i]-K,0) 
    
# Calculate the Monte Carlo estimator
C0 = exp(-r * T) * mean(payoff)
vol=sqrt(sum((payoff-mean(payoff))**2)/(I-1))
v_down=mean(payoff)-1.96*vol/sqrt(I)
v_up=mean(payoff)+1.96*vol/sqrt(I)
print("Estimated present value is %f" % C0)
print("Estimated volatility is %f" % vol)
print("Estimated  is %f" % v_down,v_up)

Estimated present value is 10.961759
Estimated volatility is 23.792350
Estimated  is 10.772927 11.705587463527088


In [130]:

def quasi_MC_DO_call_option_price(S, K, r, sigma, T, n, m, B):
    np.random.seed(0)  # 设置随机种子以保持结果的一致性
    dt = T / m  # 时间步长
    # 生成 Sobol 序列
    sobol = np.random.default_rng().random((n, m))
    sobol = np.column_stack((sobol, 1 - sobol))  # 使用维度扩展 Sobol 序列

    paths = np.zeros((n, m + 1))
    paths[:, 0] = S
    payoff = np.zeros(n)
    for i in range(1, m + 1):
        z = norm.ppf(sobol[:, i - 1])
        paths[:, i] = paths[:, i - 1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z)
    for j in range(n):
        if min(paths[j]<=B):
            payoff[j]=0 
        else:
            payoff[j]=max(paths[j,-1]-K,0) 
    return exp(-r * T) * mean(payoff)


In [132]:
# Parameters
S0 = 100; T = 0.5; K = 110; r = 0.05; sigma = 0.5
M=1000; dt = T / M; N = 10000 ; B=75
# payoff=np.zeros(I)

In [133]:
# paths[-1,0]
print(quasi_MC_DO_call_option_price(S0,K,r,sigma,T,N,M,B))

11.538516748452595


## UP and Out Option

In [23]:
from numpy import sqrt, exp, cumsum, sum, maximum, mean
from numpy.random import standard_normal
import numpy as np

# Parameters
S0 = 50; T = 0.5; K = 60; r = 0.05; sigma = 0.45
M = 252; dt = T / M; I = 100000 ; B=80
payoff=np.zeros(I)


In [24]:
# Simulate I paths with M time steps
S = S0 * exp(cumsum((r - 0.5 * sigma ** 2) * dt + sigma * sqrt(dt) * 
                          standard_normal((M, I)), axis=0 ))
for i in range(I):
    if max(S[:,i])>=B :
        payoff[i]=0 
    else :
        payoff[i]=max(S[-1:,i]-K,0) 
    
# Calculate the Monte Carlo estimator
C0 = exp(-r * T) * mean(payoff)
print("Estimated present value is %f" % C0)

Estimated present value is 0.948587


## Up and In Option

In [25]:
from numpy import sqrt, exp, cumsum, sum, maximum, mean
from numpy.random import standard_normal
import numpy as np

# Parameters
S0 = 50; T = 0.5; K = 60; r = 0.05; sigma = 0.45
M = 252; dt = T / M; I = 100000 ; B=80
payoff=np.zeros(I)


In [26]:
# Simulate I paths with M time steps
S = S0 * exp(cumsum((r - 0.5 * sigma ** 2) * dt + sigma * sqrt(dt) * 
                          standard_normal((M, I)), axis=0 ))
for i in range(I):
    if max(S[:,i])<B :
        payoff[i]=0 
    else :
        payoff[i]=max(S[-1:,i]-K,0) 
    
# Calculate the Monte Carlo estimator
C0 = exp(-r * T) * mean(payoff)
print("Estimated present value is %f" % C0)

Estimated present value is 3.413675


In [33]:
def sim_value(S,sig,r,T):
    e = np.random.normal()
    return S*np.exp((r-0.5*sig**2)*T+sig*e*np.sqrt(T))
def callpayoff(S_T,K):
    return np.exp(-r*T)*max(S_T-K,0)
S = 200
B = 175
K = 150
r = 0.05
sig = 0.2
T = 0.5
MM = 1000
R = 18.8103
N = 100.
vi = []
for i in range(2):
    S_T = sim_value(S,sig,r,T)
    print(S_T)
#     if vi <= B and S==B:
#         S_T == 0
#     else:
#         vi.append(callpayoff(S_T,K))
# price = 1/float(MM)*sum(vi)
# Variance = 1/float(MM-1)*sum((vi-price)**2)
# SE=np.sqrt(Variance/float(MM))

245.33258467955156
206.92297306710387


# Fixed Strike Lookback Option

In [17]:
from numpy import sqrt, exp, cumsum, sum, maximum, mean
from numpy.random import standard_normal
import numpy as np

# Parameters
S0 = 110; T = 1; K = 110; r = 0.35; sigma = 0.5
M = 10;dt =T/M; I = [10,100,1000,10000,100000]


In [6]:
for m in range(5):
    payoff=np.zeros(I[m])
    # Simulate I paths with M time steps
    S = S0 * exp(
        cumsum((r - 0.5 * sigma**2) * dt + sigma * sqrt(dt) * standard_normal(
            (M, I[m])),
               axis=0))
    for i in range(I[m]):
        payoff[i] = max(max(S[:, i]) - K, 0)

    # Calculate the Monte Carlo estimator
    C0 = exp(-r * T) * mean(payoff)
#     print("Estimated present value is %f" % C0)
    payoff = np.zeros(I[m])
    for i in range(I[m]):
        payoff[i] = max(K-min(S[:, i]) , 0)
    # Calculate the Monte Carlo estimator
    P0 = exp(-r * T) * mean(payoff)
    print("Estimated present value is %f" % P0)

Estimated present value is 8.721825
Estimated present value is 8.030171
Estimated present value is 8.331775
Estimated present value is 8.065519
Estimated present value is 8.051688


In [7]:

def quasi_MC_Fixed_Lookback_call_option_price(S, K, r, sigma, T, n, m):
    np.random.seed(0)  # 设置随机种子以保持结果的一致性
    dt = T / m  # 时间步长
    # 生成 Sobol 序列
    sobol = np.random.default_rng().random((n, m))
    sobol = np.column_stack((sobol, 1 - sobol))  # 使用维度扩展 Sobol 序列

    paths = np.zeros((n, m + 1))
    paths[:, 0] = S
    payoff = np.zeros(n)
    for i in range(1, m + 1):
        z = norm.ppf(sobol[:, i - 1])
        paths[:, i] = paths[:, i - 1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z)
    for j in range(n):
        payoff[j]=max(max(paths[j])-K,0) 
    return exp(-r * T) * mean(payoff)
def quasi_MC_Fixed_Lookback_put_option_price(S, K, r, sigma, T, n, m):
    np.random.seed(0)  # 设置随机种子以保持结果的一致性
    dt = T / m  # 时间步长
    # 生成 Sobol 序列
    sobol = np.random.default_rng().random((n, m))
    sobol = np.column_stack((sobol, 1 - sobol))  # 使用维度扩展 Sobol 序列

    paths = np.zeros((n, m + 1))
    paths[:, 0] = S
    payoff = np.zeros(n)
    for i in range(1, m + 1):
        z = norm.ppf(sobol[:, i - 1])
        paths[:, i] = paths[:, i - 1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z)
    for j in range(n):
        payoff[j]=max(K-min(paths[j]),0) 
    return exp(-r * T) * mean(payoff)

In [8]:
for m in range(5):
    call_price = quasi_MC_Fixed_Lookback_call_option_price(S0, K, r, sigma, T, I[m], M)
    put_price=quasi_MC_Fixed_Lookback_put_option_price(S0, K, r, sigma, T, I[m], M)
    print(call_price,put_price)

21.04127120993026 10.687317836660165
24.73737251621416 9.68219405442841
21.97358259180262 10.090405908615352
22.062366254850456 9.987651836731663
22.10853175327704 9.998224954086705


In [14]:
import math
from scipy.stats import norm

def fixed_strike_lookback_option(S, K, r, sigma, T, S_max):
    d1 = (math.log(S/K) + (r + (sigma**2)/2)*T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    d3 = d1 - (2 * r * math.sqrt(T)) / sigma
    e1 = (math.log(S/S_max) + (r + (sigma**2)/2)*T) / (sigma * math.sqrt(T))
    e2 = e1 - sigma * math.sqrt(T)
    e3 = e1 - (2 * r * math.sqrt(T)) / sigma

    if S_max <= K:
        term1 = S * norm.cdf(d1) - math.exp(-r * T) * K * norm.cdf(d2)
        term2 = math.exp(-r * T) * (sigma**2 / (2 * r)) * S * (math.exp(r * T) * norm.cdf(d1) - ((S / K)**(-2 * r / (sigma**2))) * norm.cdf(d3))
        option_price = term1 + term2
    else:
        term1 = math.exp(-r * T) * (S_max * (1 - norm.cdf(d2)) - K)
        term2 = math.exp(-r * T) * S * (sigma**2 / (2 * r)) * (math.exp(r * T) * norm.cdf(d1) - ((S / S_max)**(-2 * r / (sigma**2))) * norm.cdf(d3))
        option_price = term1 + term2

    return option_price
def fixed_strike_lookback_option_put(S, K, r, sigma, T, S_min):
    d1 = (math.log(S/K) + (r + (sigma**2)/2)*T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    d3 = d1 - (2 * r * math.sqrt(T)) / sigma
    f1 = (math.log(S/S_min) + (r + (sigma**2)/2)*T) / (sigma * math.sqrt(T))
    f2 = f1 - sigma * math.sqrt(T)
    f3 = f1 - (2 * r * math.sqrt(T)) / sigma

    if K <= S_min:
        term1 = K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
        term2 = math.exp(-r * T) * (sigma**2 / (2 * r)) * S * (-math.exp(r * T) * norm.cdf(-d1) + ((S / K)**(-2 * r / (sigma**2))) * norm.cdf(-d3))
        option_price = term1 + term2
    else:
        term1 = math.exp(-r * T) * (S_min * (norm.cdf(-f2) - 1) + K) - S * norm.cdf(-f1)
        term2 = math.exp(-r * T) * S * (sigma**2 / (2 * r)) * (-math.exp(r * T) * norm.cdf(-f1) + ((S / S_min)**(-2 * r / (sigma**2))) * norm.cdf(-f3))
        option_price = term1 + term2

    return option_price

In [18]:
# for m in range(3):
a=fixed_strike_lookback_option(S0,K,r,sigma,T,K)
# fixed_strike_lookback_option_put(100,100,0.2,0.4,0.25,100) 
b=fixed_strike_lookback_option_put(S0,K,r,sigma,T,S0)
print(a,b)

62.49656344538103 18.410713981960384


# Floating Strike Lookback Option

In [362]:
from numpy import sqrt, exp, cumsum, sum, maximum, mean
from numpy.random import standard_normal
import numpy as np

# Parameters
S0 = 100; T = 0.5; K = 110; r = 0.2; sigma = 0.4
M = 1000; dt = T / M; I = 1000 
payoff=np.zeros(I)
smin=np.zeros(I)
smax=np.zeros(I) 

In [176]:
# Simulate I paths with M time steps
S = S0 * exp(
    cumsum((r - 0.5 * sigma**2) * dt + sigma * sqrt(dt) * standard_normal(
        (M, I)),
           axis=0)) 

for i in range(I):
    payoff[i] = max(S[-1:,i]-min(S[:, i]) , 0)
    smin[i]=min(S[:, i])
    smax[i]=max(S[:, i])
MAX=mean(smax)
MIN=mean(smin)
# Calculate the Monte Carlo estimator
C0 = exp(-r * T) * mean(payoff)
print("Estimated present value is %f" % C0)
for i in range(I):
    payoff[i] = max(max(S[:, i])-S[-1:,i] , 0)
# Calculate the Monte Carlo estimator
P0 = exp(-r * T) * mean(payoff)
print("Estimated present value is %f" % P0)


Estimated present value is 16.676908
Estimated present value is 14.031043


In [177]:
import numpy as np
from scipy.stats import norm

def quasi_MC_Floating_Lookback_call_option_price(S, K, r, sigma, T, n, m):
    np.random.seed(0)  # 设置随机种子以保持结果的一致性
    dt = T / m  # 时间步长
    # 生成 Sobol 序列
    sobol = np.random.default_rng().random((n, m))
    sobol = np.column_stack((sobol, 1 - sobol))  # 使用维度扩展 Sobol 序列

    paths = np.zeros((n, m + 1))
    paths[:, 0] = S
    payoff = np.zeros(n)
    for i in range(1, m + 1):
        z = norm.ppf(sobol[:, i - 1])
        paths[:, i] = paths[:, i - 1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z)
    for j in range(n):
        payoff[j]=max(paths[j,-1]-min(paths[j]),0) 
    return exp(-r * T) * mean(payoff)

def quasi_MC_Floating_Lookback_put_option_price(S, K, r, sigma, T, n, m):
    np.random.seed(0)  # 设置随机种子以保持结果的一致性
    dt = T / m  # 时间步长
    # 生成 Sobol 序列
    sobol = np.random.default_rng().random((n, m))
    sobol = np.column_stack((sobol, 1 - sobol))  # 使用维度扩展 Sobol 序列

    paths = np.zeros((n, m + 1))
    paths[:, 0] = S
    payoff = np.zeros(n)
    for i in range(1, m + 1):
        z = norm.ppf(sobol[:, i - 1])
        paths[:, i] = paths[:, i - 1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z)
    for j in range(n):
        payoff[j]=max(max(paths[j])-paths[j,-1],0) 
    return exp(-r * T) * mean(payoff)

In [180]:
call_price = quasi_MC_Floating_Lookback_call_option_price(S0, K, r, sigma, T, I, M)

put_price=quasi_MC_Floating_Lookback_put_option_price(S0, K, r, sigma, T, I, M)

print(call_price,put_price)


16.90196151612262 14.08795839697109


In [360]:
import math

def calculate_floating_call_option_price(S, r, sigma, tau,S_min):
    d1 = (math.log(S / S_min) + (r + 0.5 * sigma**2) * tau) / (sigma * math.sqrt(tau))
    d2 = d1 - sigma * math.sqrt(tau)
    d3 = d1 - (2 * r / sigma) * math.sqrt(tau)

    N = lambda x: 0.5 * (1 + math.erf(x / math.sqrt(2)))

    option_price = S * N(d1) - S_min * math.exp(-r * tau) * N(d2) + math.exp(-r * tau) * (sigma**2 / (2 * r)) * S * ((S / S_min)**(-2 * r / sigma**2) * N(-d3) - math.exp(r * tau) * N(-d1))
    return option_price
def calculate_floating_put_option_price(S, r, sigma, tau, S_max):
    d1 = (math.log(S/S_max) + (r + (sigma**2)/2)*tau) / (sigma*math.sqrt(tau))
    d2 = d1 - sigma*math.sqrt(tau)
    d3 = d1 - (2*r/(sigma))*math.sqrt(tau)

    term1 = -S * norm.cdf(-d1)
    term2 = S_max * math.exp(-r*tau) * norm.cdf(-d2)
    term3 = math.exp(-r*tau) * (sigma**2) / (2*r) * S * (-((S/S_max)**(-2*r/(sigma**2))) * norm.cdf(d3) + math.exp(r*tau) * norm.cdf(d1))

    P_Floating = term1 + term2 + term3
    return P_Floating


In [363]:
c=calculate_floating_call_option_price(S0,r,sigma,T,S0) 
p=calculate_floating_put_option_price(S0,r,sigma,T,S0) 
print(c,p)

24.8513499774796 19.141595059637154
